In [1]:
#package installs
#E:\Python310\python.exe -m pip install --upgrade pip
#!E:\Python310\python.exe -m pip install git+https://github.com/openai/whisper.git soundfile
#!E:\Python310\Scripts\pip3.exe install ffmpeg av
#!E:\Python310\python.exe -m pip uninstall torch torchvision torchaudio -y
#!E:\Python310\python.exe -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


In [1]:
#Global variables
import socket
import os
ENV_HOSTNAME = socket.gethostname()
print('ENV_HOSTNAME:' + ENV_HOSTNAME)

#store defaults for Jacob here:
ENV_FOLDER_DATA = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data\\'
ENV_FOLDER_DATA_PROC = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data_Proc\\'
ENV_PRATT = ''


if ENV_HOSTNAME == 'JWGamingPC':
    ENV_FOLDER_DATA = 'E:\\W4732 Computer Vision\\Final Paper Data\\'
    ENV_FOLDER_DATA_PROC = 'E:\\W4732 Computer Vision\\Final Paper Data Proc\\'
    ENV_PRATT = ''

print('ENV_FOLDER_DATA:' + ENV_FOLDER_DATA)
print('ENV_PRATT:' + ENV_PRATT)

# Create folder structure
import os
os.makedirs(ENV_FOLDER_DATA_PROC + 'segmentation', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'targetdf', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'pratt', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'eps', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'clips', exist_ok=True)


ENV_HOSTNAME:JWGamingPC
ENV_FOLDER_DATA:E:\W4732 Computer Vision\Final Paper Data\
ENV_PRATT:


In [38]:

import torch.nn as nn
import torch.nn.functional as F
import torch


class JWVideoConvNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=41):
        super().__init__()
        # TODO: Initialize network layers
        

        
        self.conv1 = torch.nn.Conv3d(in_channels=in_channels, out_channels=12
                                    , kernel_size=(2,6,6), padding=3, stride=(1,2,2))
        self.conv2 = torch.nn.Conv3d(in_channels=12, out_channels=36
                                    , kernel_size=(2,6,6), padding=3, stride=(1,2,2))
        self.conv3 = torch.nn.Conv3d(in_channels=36, out_channels=72
                                     , kernel_size=(2,6,6), padding=3, stride=(1,2,2))
        
        self.lin1 = torch.nn.Linear(35640, 4096)
        self.lin2 = torch.nn.Linear(4096, 1024)
        self.lin3 = torch.nn.Linear(1024, num_classes)

        self.drop = torch.nn.Dropout(0.1)
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.LeakyReLU(0.1)
        self.maxpool = torch.nn.MaxPool3d(kernel_size=(1,2,2), stride=(1,2,2))
        #self.batchnorm12 = torch.nn.BatchNorm3d(12)
        #self.batchnorm36 = torch.nn.BatchNorm3d(32)
        
    def forward(self, vid):
        x = self.conv1(vid)
        x = self.maxpool(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.maxpool(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.maxpool(x)
        x = self.relu(x)

        x = torch.flatten(x,1)
        #new_face = torch.Tensor(list(face))
        #x = torch.cat([x,new_face] , dim=1)
        x = self.lin1(x)
        x = self.relu(x)
        x = self.drop(x)

        x = self.lin2(x)
        x = self.relu(x)
        x = self.drop(x)
        x = self.lin3(x)
        #print("fc3: ", x.shape)
        return x

a = JWVideoConvNet()
a = a.to(device='cuda')

In [39]:
## Model Training 1 ##
import pandas as pd
import json
import copy
import csv
import pickle
import torchvision
import itertools
import torch
import glob
import random



#get number after # but before space afterwards

#generate filepaths
folderpath_clipinfo = ENV_FOLDER_DATA_PROC + 'clipinfo\\'
os.makedirs(folderpath_clipinfo, exist_ok=True)
folderpath_sourcedf = ENV_FOLDER_DATA_PROC + 'sourcedf\\'
os.makedirs(folderpath_sourcedf, exist_ok=True)
folderpath_sourcefacedf = ENV_FOLDER_DATA_PROC + 'sourcefacecdf\\'
os.makedirs(folderpath_sourcefacedf, exist_ok=True)
folderpath_targetdf = ENV_FOLDER_DATA_PROC + 'targetdf\\'
folderpath_pratt = ENV_FOLDER_DATA_PROC + 'pratt\\' 
filepath_speakers = ENV_FOLDER_DATA_PROC + 'speakers.json'
#filepath_sourcedf = folderpath_sourcedf + str_epnum + '.pickle'
#filepath_sourcefacedf = folderpath_sourcefacedf + str_epnum + '.pickle'

## get overlap between sourcedf files and targetdf files
list_eps = []

#note - source is actually a dict of dict of tensors
dict_sourcedf = {}
list_sourcedf_eps = []
for path_pickle in glob.glob(folderpath_sourcefacedf + '*.pickle'):
    dict_temp = {}
    dict_temp['path'] = path_pickle
    str_basenm = os.path.basename(path_pickle)
    dict_temp['basenm'] = str_basenm
    dict_temp['str_epnum'] = str_basenm.split('.')[0]
    int_epnum = int(dict_temp['str_epnum'])
    dict_temp['int_epnum'] = int_epnum
    list_sourcedf_eps.append(int_epnum)
    dict_sourcedf[int_epnum] = dict_temp

#note - target is actually a df
dict_targetdf = {}
list_targetdf_eps = []
for path_pickle in glob.glob(folderpath_targetdf + '*.pickle'):
    dict_temp = {}
    dict_temp['path'] = path_pickle
    str_basenm = os.path.basename(path_pickle)
    dict_temp['basenm'] = str_basenm
    dict_temp['str_epnum'] = str_basenm.split('.')[0]
    int_epnum = int(dict_temp['str_epnum'])
    dict_temp['int_epnum'] = int_epnum
    list_targetdf_eps.append(int_epnum)
    dict_targetdf[int_epnum] = dict_temp

#populate matches
for src_ep in list_sourcedf_eps:
    if src_ep in list_targetdf_eps:
        list_eps.append(src_ep)

#40 data elements
list_datacols = []
list_datacols.append('m')
for i in range(40):
    list_datacols.append('d' + str(i))

#define model pieces
NUM_EPOCHS = 100
TRAINING_TYPE = 'p'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#transform=torchvision.transforms.Normalize( mean=(255*0.5), std=(255*0.5) )
optimizer = torch.optim.Adam(a.parameters(), lr=0.001, betas=(0.5, 0.999))
criterion = torch.nn.MSELoss().to(device)


##Epoch Loop
for e in range(NUM_EPOCHS):
    print('Epoch:' + str(e))
    #make a copy of the episode list to pop out 2 at a time
    list_epocheps = copy.deepcopy(list_eps)
    random.shuffle(list_epocheps)

    list_loss = []

    while len(list_epocheps) > 0:
        #load two or 3 episodes into memory at a time
        list_tempeps = []
        for i in range(3):
            if len(list_epocheps) > 0:
                list_tempeps.append(list_epocheps.pop())
        #load pratt files into memory
        dict_target = {}
        for i,tempeps in enumerate(list_tempeps):
            with open(dict_targetdf[tempeps]['path'], 'rb') as file:
                dict_target[tempeps] =  pickle.load(file)
                    
        #load video files into memory
        dict_source = {}
        for i,tempeps in enumerate(list_tempeps):
            with open(dict_sourcedf[tempeps]['path'], 'rb') as file:
                dict_temp = pickle.load(file)
                dict_source[tempeps] = dict_temp

        #create a list of eps / clips and shuffle
        listtup_epseg = []
        for key,val in dict_source.items():
            for key2,val2 in val.items():
                tup_epseg = [key,key2]
                listtup_epseg.append( tup_epseg)
        random.shuffle(listtup_epseg)

        #loop for data elements
        for tup_epseg in listtup_epseg:
            int_epnum = tup_epseg[0]
            int_segnum = tup_epseg[1]
            #feed values into the predict / loss / optimize cycle
            inputs = None
            labels = None
            if int_epnum not in dict_source.keys():
                continue
            if int_segnum not in dict_source[int_epnum].keys():
                continue
            face = dict_source[int_epnum][int_segnum][1]
            list_inputs = dict_source[int_epnum][int_segnum][0]
            #pad inputs
            inputs = []
            total_frames = 0
            bad_frames = 0
            for x in range(40):
                total_frames += 1
                if x >= len(list_inputs):
                    inputs.append( torch.empty((3,150,150), dtype=torch.int8) )
                    bad_frames += 1
                elif face[x][2] < 40 or face[x][3] < 40:
                    inputs.append( torch.empty((3,150,150), dtype=torch.int8) )
                    bad_frames += 1
                elif face[x][2] > 150 or face[x][3] > 150:
                    inputs.append( torch.empty((3,150,150), dtype=torch.int8) )
                    bad_frames += 1
                else:
                    #print('w=' + str(face[x][2]))
                    #print('h=' + str(face[x][3]))
                    #print('l=' + str((150 - face[x][2]) // 2 ))
                    #print('r=' + str(150 - face[x][2] - ((150 - face[x][2]) // 2)))
                    #print('t=' + str((150 - face[x][3]) // 2))
                    #print('b=' + str(150 - face[x][3] - ((150 - face[x][3]) // 2)))

                    inputs.append( torch.nn.functional.pad(
                        # pad(left, right, top, bottom)
                        # face = (x,y,w,h)
                        input=list_inputs[x], pad=(
                            (150 - list_inputs[x].size(dim=2)) // 2  #left
                            , 150 - list_inputs[x].size(dim=2) - ((150 - list_inputs[x].size(dim=2)) // 2) #right
                            , (150 - list_inputs[x].size(dim=1)) // 2 #top
                            , 150 - list_inputs[x].size(dim=1) - ((150 - list_inputs[x].size(dim=1)) // 2) #bottom
                            ), mode='constant', value=0
                        ))
                    #need to center the face image
            #if all the frames are bad, skip it
            if total_frames == bad_frames:
                continue
            inputs = torch.stack(inputs[:40], 0)
            #transform inputs
            #print(inputs.size())
            #torch.Size([40, 1, 360, 640])
            inputs = inputs.to(torch.float).movedim(0,1).unsqueeze(0) / 255.0
            #print(inputs.size())
            

            #check if labels exist
            label_df = dict_target[int_epnum].query('seg==' + str(int_segnum) + ' & type=="' + TRAINING_TYPE + '"' )
            if len(label_df) == 0:
                #print('No labels for ep:' + str(int_epnum) + ' seg:' + str(int_segnum))
                continue
            labels = torch.squeeze(torch.tensor(label_df[list_datacols].fillna(0).values).to(torch.float))
            #normalize labels 
            if TRAINING_TYPE == 'p':
                labels = labels / 600.0
            elif TRAINING_TYPE == 'i':
                labels = labels / 100.0
            elif TRAINING_TYPE == 's':
                labels = labels / 2
            elif TRAINING_TYPE == 'j':
                labels = labels
            elif TRAINING_TYPE == 'h':
                labels = (labels + 40) / 100
            

            inputs, labels = inputs.to(device), labels.to(device)
            #inputs_face = torch.Tensor(face[0]).to(torch.float).to(device) / 640
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = a(inputs)
            loss = criterion(outputs.squeeze(), labels.squeeze())
            list_loss.append(loss)
            loss.backward()
            optimizer.step()

        # print statistics
        print(f'Epoch={e + 1} Loss={sum(list_loss):.3f}')




    #define a function that, given a targetdf value, look up the source - if both exists, use the data to train the model

#end loop through data
#33 minutes per epoch

Epoch:0
Epoch=1 Loss=27677.734
Epoch=1 Loss=27729.807
Epoch=1 Loss=27755.123
Epoch=1 Loss=27795.746
Epoch=1 Loss=27846.916
Epoch=1 Loss=27872.885
Epoch=1 Loss=27897.428
Epoch=1 Loss=8006230081536.000
Epoch=1 Loss=8008265891840.000
Epoch=1 Loss=8009481715712.000
Epoch=1 Loss=8009840852992.000
Epoch=1 Loss=8010230398976.000
Epoch=1 Loss=8010391355392.000
Epoch=1 Loss=8010418094080.000
Epoch=1 Loss=8010446929920.000
Epoch:1
Epoch=2 Loss=26985784.000
Epoch=2 Loss=38758596.000
Epoch=2 Loss=281938272.000
Epoch=2 Loss=293566848.000
Epoch=2 Loss=294866080.000
Epoch=2 Loss=295806368.000
Epoch=2 Loss=296290464.000
Epoch=2 Loss=297216416.000
Epoch=2 Loss=4209926471680.000
Epoch=2 Loss=4221979852800.000
Epoch=2 Loss=4457795420160.000
Epoch=2 Loss=4465388158976.000
Epoch=2 Loss=4467050676224.000
Epoch=2 Loss=4468408582144.000
Epoch=2 Loss=4469012561920.000
Epoch:2
Epoch=3 Loss=1139617024.000
Epoch=3 Loss=1482624128.000
Epoch=3 Loss=1594083456.000
Epoch=3 Loss=1657926912.000
Epoch=3 Loss=2078622720.

In [5]:
#save model down for future use / training
torch.save(a.state_dict(), ENV_FOLDER_DATA_PROC + 'charts\\' + "JWPitchModv04Fin.pt")

In [ ]:
#have some metric to show 

<H1> Appendix </H1>

In [2]:
## Model Training 1 ##
import json
import copy
import csv
import pickle
import itertools
import glob
import random



#get number after # but before space afterwards

#generate filepaths
folderpath_clipinfo = ENV_FOLDER_DATA_PROC + 'clipinfo\\'
os.makedirs(folderpath_clipinfo, exist_ok=True)
folderpath_sourcedf = ENV_FOLDER_DATA_PROC + 'sourcedf\\'
os.makedirs(folderpath_sourcedf, exist_ok=True)
folderpath_sourcefacedf = ENV_FOLDER_DATA_PROC + 'sourcefacecdf\\'
os.makedirs(folderpath_sourcefacedf, exist_ok=True)
folderpath_targetdf = ENV_FOLDER_DATA_PROC + 'targetdf\\'
folderpath_pratt = ENV_FOLDER_DATA_PROC + 'pratt\\' 
filepath_speakers = ENV_FOLDER_DATA_PROC + 'speakers.json'
#filepath_sourcedf = folderpath_sourcedf + str_epnum + '.pickle'
#filepath_sourcefacedf = folderpath_sourcefacedf + str_epnum + '.pickle'

## get overlap between sourcedf files and targetdf files
list_eps = []

#note - source is actually a dict of dict of tensors
dict_sourcedf = {}
list_eps = []
for path_pickle in glob.glob(folderpath_sourcefacedf + '*.pickle'):
    dict_temp = {}
    dict_temp['path'] = path_pickle
    str_basenm = os.path.basename(path_pickle)
    dict_temp['basenm'] = str_basenm
    dict_temp['str_epnum'] = str_basenm.split('.')[0]
    int_epnum = int(dict_temp['str_epnum'])
    dict_temp['int_epnum'] = int_epnum
    list_eps.append(int_epnum)
    dict_sourcedf[int_epnum] = dict_temp

list_output_frameloc = []

while len(list_eps) > 0:
    #load two or 3 episodes into memory at a time
    list_tempeps = []
    for i in range(3):
        if len(list_eps) > 0:
            list_tempeps.append(list_eps.pop())

                
    #load video files into memory
    dict_source = {}
    for i,tempeps in enumerate(list_tempeps):
        with open(dict_sourcedf[tempeps]['path'], 'rb') as file:
            dict_temp = pickle.load(file)
            dict_source[tempeps] = dict_temp

    #create a list of eps / clips and shuffle
    listtup_epseg = []
    for key,val in dict_source.items():
        for key2,val2 in val.items():
            tup_epseg = [key,key2]
            listtup_epseg.append( tup_epseg)
    random.shuffle(listtup_epseg)

    #loop for data elements
    for tup_epseg in listtup_epseg:
        int_epnum = tup_epseg[0]
        int_segnum = tup_epseg[1]
        #feed values into the predict / loss / optimize cycle
        inputs = None
        if int_epnum not in dict_source.keys():
            continue
        if int_segnum not in dict_source[int_epnum].keys():
            continue
        inputs = dict_source[int_epnum][int_segnum]

        frames = inputs[0]
        frame_loc = inputs[1]
        list_output_frameloc.append(frame_loc)






In [12]:
length = []
width = []
count_face = 0
count_noface = 0
for x in list_output_frameloc:
    for y in x:
        if (y[2] == 0 and y[3] == 0) or y[2] > 150 :
            count_noface += 1
        else:
            count_face += 1
            width.append(y[2])
            length.append(y[3])
        

In [14]:
print(min(length))
print(max(length))
print(min(width))
print(max(width))
print(count_face)
print(count_noface)
print(1762179.0 / (297210 + 1762179)) #85.5% face hitrate for 150x150

44
150
44
150
1762179
297210
0.8556804955256146
